In [9]:
print(__doc__)

from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale


Automatically created module for IPython interactive environment


In [10]:
np.random.seed(42)

digits = load_digits()
data = scale(digits.data)

n_samples, n_features = data.shape
n_digits = len(np.unique(digits.target))
labels = digits.target

sample_size = 300

print("n_digits: %d, \t n_samples: %d, \t n_features: %d"
     %(n_digits, n_samples, n_features))

n_digits: 10, 	 n_samples: 1797, 	 n_features: 64


In [13]:
def bench_k_means(estimator, name, data):
    to = time() 
    estimator.fit(data) 
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
        % (name, (time() - to), estimator.inertia_,
        metrics.homogeneity_score(labels, estimator.labels_),
        metrics.completeness_score(labels, estimator.labels_),
        metrics.v_measure_score(labels, estimator.labels_),
        metrics.adjusted_rand_score(labels, estimator.labels_),
        metrics.adjusted_mutual_info_score(labels, estimator.labels_), 
        metrics.silhouette_score(data, estimator.labels_, metric=
                            'euclidean', sample_size = sample_size)))
    
bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
                name="k-means++", data=data) 
bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
                name="random", data=data)
pca = PCA(n_components=n_digits).fit(data) 
bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
                name="PCA based", data=data)

k-means++	0.24s	69979	0.607	0.657	0.631	0.482	0.628	0.137
random   	0.18s	69668	0.676	0.715	0.695	0.564	0.692	0.149
PCA based	0.03s	70804	0.671	0.697	0.684	0.561	0.680	0.127
